In [ ]:
!pip install medmnist
!pip install fastMONAI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=e5633c287bfae909560a13f7730b7064af66dcd6d0bd4b2feace5649d96d3f6d
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.3 MB/s eta 0:00

In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import nibabel as nib
import gzip

import medmnist
from medmnist import INFO, Evaluator

from fastMONAI.vision_all import *

In [ ]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.1.0 @ https://github.com/MedMNIST/MedMNIST/


# We check a 3D dataset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
BATCH_SIZE = 128

In [ ]:
data_flag = 'organmnist3d'
download = True

info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train',  download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/32657407 [00:00<?, ?it/s]

In [ ]:
info

{'python_class': 'OrganMNIST3D',
 'description': 'The source of the OrganMNIST3D is the same as that of the Organ{A,C,S}MNIST. Instead of 2D images, we directly use the 3D bounding boxes and process the images into 28×28×28 to perform multi-class classification of 11 body organs. The same 115 and 16 CT scans as the Organ{A,C,S}MNIST from the source training set are used as training and validation set, respectively, and the same 70 CT scans as the Organ{A,C,S}MNIST from the source test set are treated as the test set.',
 'url': 'https://zenodo.org/record/6496656/files/organmnist3d.npz?download=1',
 'MD5': '21f0a239e7f502e6eca33c3fc453c0b6',
 'task': 'multi-class',
 'label': {'0': 'liver',
  '1': 'kidney-right',
  '2': 'kidney-left',
  '3': 'femur-right',
  '4': 'femur-left',
  '5': 'bladder',
  '6': 'heart',
  '7': 'lung-right',
  '8': 'lung-left',
  '9': 'spleen',
  '10': 'pancreas'},
 'n_channels': 1,
 'n_samples': {'train': 972, 'val': 161, 'test': 610},
 'license': 'CC BY 4.0'}

In [ ]:
x, y = train_dataset[0]

print(x.shape, y.shape)
print(x.squeeze().shape, y.squeeze().shape)
print(type(x), type(y))

(1, 28, 28, 28) (1,)
(28, 28, 28) ()
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:

#functions for converting 3D arrays to a NIfTI image and back to a 3D array

def createNiftiImage(img_arr):
  return nib.Nifti1Image(img_arr, affine=np.eye(4))
def getNiftiData(niftiImage):
  return niftiImage.get_fdata()
def zipImage(image: nib.Nifti1Image, filename: str):
    # Save Nifti image to temporary file
    temp_filename = f"{filename}.temp"
    nib.save(image, temp_filename)
    # Compress file using gzip
    with open(temp_filename, "rb") as f_in:
        with gzip.GzipFile(filename, "wb") as f_out:
            f_out.writelines(f_in)
    # Delete temporary file
    os.remove(temp_filename)

In [ ]:
# Creating the arrays data_array, label_array and text_label_array from train_dataset:


labels = ['liver','kidney-right','kidney-left','femur-right','femur-left','bladder','heart','lung-right','lung-left','spleen','pancreas']
image_data = []
nifti_images = [] 
label_values = []
label_text = []
file_names = []
index = 0
for data in train_dataset:
  imageData,label = data
  image = createNiftiImage(imageData.squeeze())

  image_data.append(imageData.squeeze())
  nifti_images.append(image)
  label_values.append(label.squeeze().item())
  label_text.append(labels[label.squeeze()])

  # Save Nifti image as gzip-compressed Nifti file
  path = ".zipImages/"
  if not os.path.exists(path):
    os.makedirs(path)
  filename = path+"img"+str(index)+".nii.gz"
  zipImage(image,filename)

  index = index + 1
  file_names.append(filename)

ImageFileError: ignored

In [ ]:
# Checking shape and type of arrays

print(np.shape(image_data), np.shape(nifti_images), np.shape(label_text), np.shape(label_values))
print(type(image_data), type(nifti_images), type(label_text), type(label_values))

(972, 28, 28, 28) (972,) (972,) (972,)
<class 'list'> <class 'list'> <class 'list'> <class 'list'>


In [ ]:
print(np.shape(image_data[0]), np.shape(nifti_images[0]), np.shape(label_text[0]), np.shape(label_values[0]))
print(type(image_data[0]), type(nifti_images[0]), type(label_text[0]), type(label_values[0]))

(28, 28, 28) (28, 28, 28) () ()
<class 'numpy.ndarray'> <class 'nibabel.nifti1.Nifti1Image'> <class 'str'> <class 'int'>


In [ ]:
# Putting the three arrays in a dataframe:

df = pd.DataFrame({'image':file_names,'image_data':image_data,'label_text':label_text,'label_val':label_values})

In [ ]:
# Checking shape and type of df content

print(np.shape(df['image'][0]),np.shape(df['image_data'][0]),np.shape(df['label_text'][0]),np.shape(df['label_val'][0]))
print(type(df['image'][0]),type(df['image_data'][0]),type(df['label_text'][0]),type(df['label_val'][0]))

() (28, 28, 28) () ()
<class 'str'> <class 'numpy.ndarray'> <class 'str'> <class 'numpy.int64'>


In [ ]:
import matplotlib.pyplot as plt
# Making functions to visualy represent a datapoint

def showNiftiImage(image):
  image_data = getNiftiData(image)
  image_count = image_data.shape[0]
  for i in range(image_count):
    plt.subplot(image_count,image_count ,i + 1)
    plt.imshow(image_data[:,:,i])
    plt.gcf().set_size_inches(60, 60)
    if (i+1) % 7 == 0:
      plt.show()

def showDataFrame(df,index):
  print(str(df['label_val'][index])+' : '+str(df['label_text'][index]))
  showNiftiImage(df['image'][index])

In [ ]:
showDataFrame(df,0)

3 : femur-right


AttributeError: ignored

FASTMonai

In [ ]:
item_tfms = [ZNormalization(), PadOrCrop(size=28), IntToFloatTensor()] # Add IntToFloatTensor() transform to convert tensor to float

In [ ]:
dls = MedImageDataLoaders.from_df(df, fn_col='image_data', label_col='label_val', 
                                  resample=1, item_tfms=item_tfms, bs=4, batch_tfms=None)

In [ ]:
dls = MedImageDataLoaders.from_df(df, 
                                  fn_col='image', 
                                  label_col='label_val', 
                                  resample=1, 
                                  item_tfms=[ZNormalization(), PadOrCrop(size=28), IntToFloatTensor()], 
                                  bs=4)

RuntimeError: ignored

In [ ]:
dls.show_batch(figsize=(5,5),anatomical_plane=2)

AttributeError: ignored